In [23]:
from dotenv import load_dotenv
load_dotenv('/Users/srishtysuman/.env')

True

In [21]:
from llama_index import SimpleDirectoryReader
from llama_index import ServiceContext
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
import cohere
import os
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

In [4]:
# load documents
documents = SimpleDirectoryReader("./data/srishty").load_data()
for i,n in enumerate(documents):
    print(n)
    break

Doc ID: e9bc617f-9c26-4d47-be08-a41bd4b31ca9
Text: What I Worked On  February 2021  Before college the two main
things I worked on, outside of school, were writing and programming. I
didn't write essays. I wrote what beginning writers were supposed to
write then, and probably still are: short stories. My stories were
awful. They had hardly any plot, just characters with strong feelings,
which I ...


ServiceContext to divide the lengthy single document into several smaller chunks with some overlap. 
  Service Context container.

    The service context container is a utility container for LlamaIndex
    index and query classes. It contains the following:
    - llm_predictor: BaseLLMPredictor
    - prompt_helper: PromptHelper
    - embed_model: BaseEmbedding
    - node_parser: NodeParser
    - llama_logger: LlamaLogger (deprecated)
    - callback_manager: CallbackManager

In [5]:
service_context = ServiceContext.from_defaults(chunk_size=512, chunk_overlap=64)

create the nodes out of the generated documents.

In [6]:
node_parser = service_context.node_parser
node_parser

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x1926ebe50>, id_func=<function default_id_func at 0x19252aac0>, chunk_size=512, chunk_overlap=64, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [7]:
nodes = node_parser.get_nodes_from_documents(documents)
nodes

[TextNode(id_='0764cbf8-9849-4d44-8cb0-6c7a2c57f607', embedding=None, metadata={'file_path': 'data/srishty/srishty_essay.txt', 'file_name': 'srishty_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-02-07', 'last_modified_date': '2024-02-07', 'last_accessed_date': '2024-02-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e9bc617f-9c26-4d47-be08-a41bd4b31ca9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'data/srishty/srishty_essay.txt', 'file_name': 'srishty_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-02-07', 'last_modified_date': '2024-02-07', 'last_accessed_date': '2024-02-07'}, hash='bc8db5c6ef2a27bff50051f05ebe0fe

The nodes must be stored in a vector store database to enable easy access. 

DeepLakeVectorStore.

    In this vector store we store the text, its embedding and
    a few pieces of its metadata in a deeplake dataset. 
    create an empty dataset when given a path. 

In [8]:
my_activeloop_org_id = "srishtysuman2919"
my_activeloop_dataset_name = "rag_llamaIndex_srishty"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

In [9]:
# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

Your Deep Lake dataset has been successfully created!


StorageContext: 

        A utility container for storing nodes, indices, and vectors. It contains the following:
            - docstore: BaseDocumentStore
            - index_store: BaseIndexStore
            - vector_store: VectorStore
            - graph_store: GraphStore

VectorStoreIndex takes in the nodes along with links to the database and uploads the data to the cloud. Essentially, it constructs the index and generates embeddings for each segment.

In [10]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.docstore.add_documents(nodes)

In [11]:
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

Uploading data to deeplake dataset.


100%|██████████| 40/40 [00:11<00:00,  3.51it/s]


\

Dataset(path='hub://srishtysuman2919/rag_llamaIndex_srishty', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (40, 1)      str     None   
 metadata     json      (40, 1)      str     None   
 embedding  embedding  (40, 1536)  float32   None   
    id        text      (40, 1)      str     None   


In [12]:
query_engine = vector_index.as_query_engine(streaming=True, similarity_top_k=10)

In [13]:
streaming_response = query_engine.query(
    "What does Paul Graham do?",
)
streaming_response.print_response_stream()

Paul Graham is a writer and programmer who has a background in computer science. He has written essays on various topics and has published them online. Additionally, he has worked on building software and has a passion for Lisp hacking.

Sub Question Query Engine

    Sub Question Query Engine, a more sophisticated querying method, can be employed to address the challenge of responding to complex queries. This engine can generate several sub-questions from the user's main question, answer each separately, and then compile the responses to construct the final answer. First, we must modify the previous query engine by removing the streaming flag, which conflicts with this technique.


In [14]:
query_engine = vector_index.as_query_engine(similarity_top_k=10)

In [16]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="pg_essay",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

We register the created query_engine as a tool by employing the QueryEngineTool class and compose metadata (description) for it. It is done to inform the framework about this tool's function and enable it to select the most suitable tool for a given task, especially when multiple tools are available. Then, the combination of the tools we declared earlier and the service context, which was previously defined, can be used to initialize the SubQuestionQueryEngine object.

In [19]:
response = query_engine.query(
    "How was Paul Grahams life different before, during, and after YC?"
)
print( ">>> The final response:\n", response )

Generated 3 sub questions.
[pg_essay] Q: What did Paul Graham work on before YC?
[pg_essay] Q: What did Paul Graham work on during YC?
[pg_essay] Q: What did Paul Graham work on after YC?
[pg_essay] A: Before YC, Paul Graham worked on a project called Bel, which was a new version of Arc. He also worked on Hacker News, which was originally called Startup News. Additionally, he wrote essays and worked on other projects with his colleagues Robert and Trevor.
[pg_essay] A: During YC, Paul Graham worked on various projects. He wrote all of YC's internal software in Arc and continued to work on it, although he gradually stopped as the infrastructure depending on it grew. He also worked on writing essays and helping the startups in the YC program.
[pg_essay] A: After leaving Y Combinator (YC), Paul Graham worked on various projects. He mentioned that he wanted to do something completely different, so he decided to focus on painting. He spent most of 2014 painting and improving his skills. How

    Custom Retriever Engine
    https://docs.llamaindex.ai/en/stable/examples/low_level/fusion_retriever.html

Reranking

In [24]:
# Get your cohere API key on: www.cohere.com
co = cohere.Client()

# Example query and passages
query = "What is the capital of the United States?"
documents = [
   "Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.",
   "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
   "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
   "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. ",
   "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
   "North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of government is Bismarck."
   ]

In [25]:
results = co.rerank(query=query, documents=documents, top_n=3, model='rerank-english-v2.0')
for idx, r in enumerate(results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Document: {r.document['text']}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

Document Rank: 1, Document Index: 3
Document: Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. 
Relevance Score: 0.98


Document Rank: 2, Document Index: 1
Document: The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.
Relevance Score: 0.30


Document Rank: 3, Document Index: 4
Document: Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.
Relevance Score: 0.28




In [26]:
cohere_rerank = CohereRerank(top_n=2)

In [27]:
query_engine = vector_index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)

response = query_engine.query(
    "What did Sam Altman do in this essay?",
)
print(response)

Sam Altman was asked if he wanted to be the president of Y Combinator (YC) and initially said no. However, after persistent persuasion, he agreed to take over as the president starting with the winter 2014 batch. He was given the opportunity to learn the job and gradually took over running YC from the original founders.



    Rerank computes a relevance score for the query and each document and returns a sorted list from the most to the least relevant document.